In [ ]:
import re
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import TFRobertaModel, RobertaTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load RoBERTa tokenizer and model
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
roberta_model = TFRobertaModel.from_pretrained(model_name)

# Function to clean and preprocess text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"http\S+|www\S+", "", text)  # Remove URLs
    text = re.sub(r"@\w+", "", text)  # Remove mentions (@username)
    text = re.sub(r"[^\w\s]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces

    return text

# Function to preprocess text (tokenization)
def preprocess_text(texts, max_length=128):
    cleaned_texts = [clean_text(text) for text in texts]  # Apply cleaning
    tokens = tokenizer(cleaned_texts, max_length=max_length, padding='max_length', truncation=True, return_tensors="tf")
    return {'input_ids': tokens['input_ids'], 'attention_mask': tokens['attention_mask']}

# Load dataset
df = pd.read_csv("/kaggle/input/twitter-us-airline/Twitter_US_Airline/Tweets.csv")  # Ensure file exists

# Encode sentiment labels (Negative=0, Neutral=1, Positive=2)
label_encoder = LabelEncoder()
df['sentiment_encoded'] = label_encoder.fit_transform(df['airline_sentiment'])

# Split data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['sentiment_encoded'], test_size=0.2, random_state=42)

# Tokenize and preprocess text data
train_inputs = preprocess_text(list(train_texts))
test_inputs = preprocess_text(list(test_texts))

# Convert labels to numpy arrays
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

# Define Attention Layer
class Attention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W = tf.keras.layers.Dense(units)
        self.b = tf.keras.layers.Dense(units)
        self.u = tf.keras.layers.Dense(1)

    def call(self, inputs):
        score = tf.nn.tanh(self.W(inputs) + self.b(inputs))
        attention_weights = tf.nn.softmax(self.u(score), axis=1)
        context_vector = attention_weights * inputs
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector

# Define Ensemble Model: RoBERTa + LSTM + BiLSTM + GRU + Attention
class RobertaEnsembleModel(tf.keras.Model):
    def __init__(self, roberta_model, lstm_units=128, num_classes=3):
        super(RobertaEnsembleModel, self).__init__()
        self.roberta = roberta_model
        
        # Recurrent layers
        self.lstm = tf.keras.layers.LSTM(lstm_units, return_sequences=True)
        self.bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences=True))
        self.gru = tf.keras.layers.GRU(lstm_units, return_sequences=True)
        
        # Attention layers
        self.lstm_attention = Attention(lstm_units)
        self.bilstm_attention = Attention(lstm_units * 2)  # Bidirectional doubles the units
        self.gru_attention = Attention(lstm_units)

        # Fully connected layers
        self.concat = tf.keras.layers.Concatenate()
        self.dropout = tf.keras.layers.Dropout(0.3)
        self.dense = tf.keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        roberta_outputs = self.roberta(inputs)[0]
        lstm_out = self.lstm(roberta_outputs)
        bilstm_out = self.bilstm(roberta_outputs)
        gru_out = self.gru(roberta_outputs)

        # Apply Attention to each recurrent layer
        lstm_attn = self.lstm_attention(lstm_out)
        bilstm_attn = self.bilstm_attention(bilstm_out)
        gru_attn = self.gru_attention(gru_out)

        # Merge outputs
        merged = self.concat([lstm_attn, bilstm_attn, gru_attn])
        dropout_out = self.dropout(merged)
        return self.dense(dropout_out)

# Initialize Model
num_classes = len(label_encoder.classes_)  # 3 classes (Negative, Neutral, Positive)
model = RobertaEnsembleModel(roberta_model, lstm_units=128, num_classes=num_classes)

# Compile Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Train Model
model.fit(train_inputs, train_labels, epochs=50, batch_size=8, validation_data=(test_inputs, test_labels))

# Evaluate Model
loss, accuracy = model.evaluate(test_inputs, test_labels)
print(f"Test Accuracy: {accuracy:.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'roberta.embeddings.position_ids', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Epoch 1/50
1464/1464 [==============================] - 303s 177ms/step - loss: 0.4867 - accuracy: 0.8089 - val_loss: 0.4543 - val_accuracy: 0.8186
Epoch 2/50
1464/1464 [==============================] - 254s 174ms/step - loss: 0.3523 - accuracy: 0.8671 - val_loss: 0.4016 - val_accuracy: 0.8504
Epoch 3/50
1464/1464 [==============================] - 254s 173ms/step - loss: 0.2623 - accuracy: 0.9045 - val_loss: 0.4381 - val_accuracy: 0.8487
Epoch 4/50
1464/1464 [==============================] - 254s 173ms/step - loss: 0.1902 - accuracy: 0.9299 - val_loss: 0.5407 - val_accuracy: 0.8487
Epoch 5/50
1464/1464 [==============================] - 254s 173ms/step - loss: 0.1370 - accuracy: 0.9528 - val_loss: 0.5880 - val_accuracy: 0.8374
Epoch 6/50
1464/1464 [==============================] - 254s 173ms/step - loss: 0.1122 - accuracy: 0.9606 - val_loss: 0.6115 - val_accuracy: 0.8354
Epoch 7/50
1464/1464 [==============================] - 254s 173ms/step - loss: 0.0926 - accuracy: 0.9678 - val_

In [ ]:
import re
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import TFRobertaModel, RobertaTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load RoBERTa tokenizer and model
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
roberta_model = TFRobertaModel.from_pretrained(model_name)

# Function to clean and preprocess text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"http\S+|www\S+", "", text)  # Remove URLs
    text = re.sub(r"@\w+", "", text)  # Remove mentions (@username)
    text = re.sub(r"[^\w\s]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces

    return text

# Function to preprocess text (tokenization)
def preprocess_text(texts, max_length=128):
    cleaned_texts = [clean_text(text) for text in texts]  # Apply cleaning
    tokens = tokenizer(cleaned_texts, max_length=max_length, padding='max_length', truncation=True, return_tensors="tf")
    return {'input_ids': tokens['input_ids'], 'attention_mask': tokens['attention_mask']}

# Load dataset
df = pd.read_csv("/kaggle/input/imdb-50k-movies/IMDb_50K_movies/IMDB Dataset.csv")  # Ensure file exists

# Encode sentiment labels (Negative=0, Neutral=1, Positive=2)
label_encoder = LabelEncoder()
df['sentiment_encoded'] = label_encoder.fit_transform(df['sentiment'])

# Split data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['review'], df['sentiment_encoded'], test_size=0.2, random_state=42)

# Tokenize and preprocess text data
train_inputs = preprocess_text(list(train_texts))
test_inputs = preprocess_text(list(test_texts))

# Convert labels to numpy arrays
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

# Define Attention Layer
class Attention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W = tf.keras.layers.Dense(units)
        self.b = tf.keras.layers.Dense(units)
        self.u = tf.keras.layers.Dense(1)

    def call(self, inputs):
        score = tf.nn.tanh(self.W(inputs) + self.b(inputs))
        attention_weights = tf.nn.softmax(self.u(score), axis=1)
        context_vector = attention_weights * inputs
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector

# Define Ensemble Model: RoBERTa + LSTM + BiLSTM + GRU + Attention
class RobertaEnsembleModel(tf.keras.Model):
    def __init__(self, roberta_model, lstm_units=128, num_classes=3):
        super(RobertaEnsembleModel, self).__init__()
        self.roberta = roberta_model
        
        # Recurrent layers
        self.lstm = tf.keras.layers.LSTM(lstm_units, return_sequences=True)
        self.bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences=True))
        self.gru = tf.keras.layers.GRU(lstm_units, return_sequences=True)
        
        # Attention layers
        self.lstm_attention = Attention(lstm_units)
        self.bilstm_attention = Attention(lstm_units * 2)  # Bidirectional doubles the units
        self.gru_attention = Attention(lstm_units)

        # Fully connected layers
        self.concat = tf.keras.layers.Concatenate()
        self.dropout = tf.keras.layers.Dropout(0.3)
        self.dense = tf.keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        roberta_outputs = self.roberta(inputs)[0]
        lstm_out = self.lstm(roberta_outputs)
        bilstm_out = self.bilstm(roberta_outputs)
        gru_out = self.gru(roberta_outputs)

        # Apply Attention to each recurrent layer
        lstm_attn = self.lstm_attention(lstm_out)
        bilstm_attn = self.bilstm_attention(bilstm_out)
        gru_attn = self.gru_attention(gru_out)

        # Merge outputs
        merged = self.concat([lstm_attn, bilstm_attn, gru_attn])
        dropout_out = self.dropout(merged)
        return self.dense(dropout_out)

# Initialize Model
num_classes = len(label_encoder.classes_)  # 3 classes (Negative, Neutral, Positive)
model = RobertaEnsembleModel(roberta_model, lstm_units=128, num_classes=num_classes)

# Compile Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Train Model
model.fit(train_inputs, train_labels, epochs=50, batch_size=8, validation_data=(test_inputs, test_labels))

# Evaluate Model
loss, accuracy = model.evaluate(test_inputs, test_labels)
print(f"Test Accuracy: {accuracy:.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.embeddings.position_ids', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Epoch 1/50
5000/5000 [==============================] - 901s 172ms/step - loss: 0.2905 - accuracy: 0.8765 - val_loss: 0.2256 - val_accuracy: 0.9085
Epoch 2/50
5000/5000 [==============================] - 856s 171ms/step - loss: 0.2025 - accuracy: 0.9193 - val_loss: 0.2344 - val_accuracy: 0.9073
Epoch 3/50
5000/5000 [==============================] - 857s 171ms/step - loss: 0.1473 - accuracy: 0.9434 - val_loss: 0.2580 - val_accuracy: 0.9099
Epoch 4/50
5000/5000 [==============================] - 856s 171ms/step - loss: 0.1085 - accuracy: 0.9611 - val_loss: 0.2799 - val_accuracy: 0.9094
Epoch 5/50
5000/5000 [==============================] - 857s 171ms/step - loss: 0.0827 - accuracy: 0.9700 - val_loss: 0.3300 - val_accuracy: 0.9090
Epoch 6/50
5000/5000 [==============================] - 857s 171ms/step - loss: 0.0640 - accuracy: 0.9780 - val_loss: 0.3335 - val_accuracy: 0.9002
Epoch 7/50
5000/5000 [==============================] - 857s 171ms/step - loss: 0.0551 - accuracy: 0.9802 - val_